In [1]:
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


cpu


In [2]:
from src.config import config

In [3]:
from src.input_pipeline import create_dataloaders

train_loader, val_loader = create_dataloaders()

In [4]:
from src.models import MPCTransformer
model_config = config['model']

model = MPCTransformer(**model_config)

/home/jannik/miniconda3/lib/python3.10/site-packages/jax/_src/api_util.py:190: SyntaxWarning: Jitted function has static_argnums=(0, 1, 9), but only accepts 8 positional arguments. This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has {argnums_name}={argnums}, "


In [5]:
# from src.utils import get_model_output
# from src.input_pipeline import create_dataset
# from src.train import create_train_state
# import jax

# init_rng = {'params': jax.random.PRNGKey(0), 'dropout': jax.random.PRNGKey(1)}
# state = create_train_state(model, 
#                            init_rng, 
#                            config['training']['learning_rate'], 
#                            config['training']['momentum'])
# del init_rng


# dataset = create_dataset()

# indices = range(50,200)
# commands = get_model_output(model, state, dataset, indices)

# import matplotlib.pyplot as plt
# plt.plot(commands['target'], label='u_x (target)')
# plt.plot(commands['pred'], label='u_x (predicted)')
# plt.legend()
# plt.show()

In [6]:
from src.train import train

num_epochs = 10

state, metrics_history = train(model, num_epochs, train_loader, val_loader)

Epoch: 1, Train Loss: 0.003934, Val Loss: 0.003061, 
Epoch: 2, Train Loss: 0.001462, Val Loss: 0.000979, 


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(metrics_history['train_loss'], label='train_loss')
plt.plot(metrics_history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
import torch
from src.input_pipeline import create_dataset
from src.utils import get_model_output
from tqdm import tqdm

dataset = create_dataset()
indices = range(len(dataset))

commands = get_model_output(model, state, dataset, indices)

plt.plot(commands['target'], label='u_x (target)')
plt.plot(commands['pred'], label='u_x (predicted)')
plt.legend()
plt.show()

In [ ]:
from flax.training import train_state, checkpoints
import jax

CKPT_DIR = '../checkpoints/'

checkpoints.save_checkpoint(ckpt_dir=CKPT_DIR, target=state, step=0)
restored_state = checkpoints.restore_checkpoint(ckpt_dir=CKPT_DIR, target=state)

assert jax.tree_util.tree_all(jax.tree_map(lambda x, y: (x == y).all(), state.params, restored_state.params))